# Maps SARS-CoV-2 Mutations to 3D Protein Structures
[Work in progress]

This notebook maps mutation frequency of SARS-CoV-2 strains onto 3D protein structures in the [Protein Data Bank](https://www.wwpdb.org/). SARS-CoV2-2 strains and their mutations have been aggregated in the [COVID-19-Net Knowledge Graph](https://github.com/covid-19-net/covid-19-community).

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, FloatSlider, SelectMultiple
from py2neo import Graph
import py3Dmol

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
output_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures

#### Connect to COVID-19-Community Knowledge Graph
[COVID-19-Net Knowledge Graph](https://github.com/covid-19-net/covid-19-community)

In [4]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### Get list of SARS-CoV-2 proteins

In [5]:
pathogen_taxonomy_id = 'taxonomy:2697049'

In [6]:
query = """
MATCH (g:Gene)-[ENCODES]->(p:Protein{taxonomyId: $pathogen_taxonomy_id})
      -[:HAS_TERTIARY_STRUCTURE]->(:Chain)-[:IS_PART_OF_STRUCTURE]->(s:Structure)
RETURN p.name AS protein, p.accession as accession, p.proId as proId, p.sequence as sequence, s.description AS description
ORDER BY protein
"""

In [7]:
structures = graph.run(query, pathogen_taxonomy_id=pathogen_taxonomy_id).to_data_frame()

In [8]:
proteins = structures[['protein', 'accession', 'proId', 'sequence']].drop_duplicates()

In [9]:
proteins.head(50)

,protein,accession,proId,sequence
0,2'-O-methyltransferase,uniprot:P0DTD1,uniprot.chain:PRO_0000449633,SSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAK...
23,3C-like proteinase,uniprot:P0DTD1,uniprot.chain:PRO_0000449623,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
240,Envelope small membrane protein,uniprot:P0DTC4,uniprot.chain:PRO_0000449651,MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNI...
245,Helicase,uniprot:P0DTD1,uniprot.chain:PRO_0000449630,AVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVN...
351,Host translation inhibitor nsp1,uniprot:P0DTD1,uniprot.chain:PRO_0000449619,MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...
364,Non-structural protein 10,uniprot:P0DTD1,uniprot.chain:PRO_0000449628,AGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLC...
389,Non-structural protein 3,uniprot:P0DTD1,uniprot.chain:PRO_0000449621,APTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVEL...
448,Non-structural protein 7,uniprot:P0DTD1,uniprot.chain:PRO_0000449625,SKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTT...
472,Non-structural protein 8,uniprot:P0DTD1,uniprot.chain:PRO_0000449626,AIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAK...
507,Non-structural protein 9,uniprot:P0DTD1,uniprot.chain:PRO_0000449627,NNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLS...


In [10]:
protein_list = structures['protein'].unique()

In [11]:
protein_widget = widgets.Dropdown(options=protein_list, description='Select protein:', value='Spike glycoprotein',
                                  style={'description_width': 'initial'}, )

### Select SARS-CoV-2 Protein

In [12]:
display(protein_widget)

Dropdown(description='Select protein:', index=17, options=("2'-O-methyltransferase", '3C-like proteinase', 'En…

In [13]:
protein_name = protein_widget.value
print('Protein name:', protein_name)

Protein name: Spike glycoprotein


In [14]:
subset = structures.query(f'protein == "{protein_name}"')

In [15]:
descriptions = list(subset['description'].unique())
descriptions.sort()
descriptions.insert(0, 'All')

In [16]:
description_widget = SelectMultiple(options=descriptions, value=['All'], description='Select subset', 
                                    layout={'width': 'max-content'}, style={'description_width': 'initial'}, 
                                    rows=len(descriptions), disabled=False)

### Select a subset of structures to be analyzed based on PDB structure description
The default is All structures. Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.

In [17]:
display(description_widget)

SelectMultiple(description='Select subset', index=(0,), layout=Layout(width='max-content'), options=('All', 'A…

In [18]:
print("Selected subset:")
for value in description_widget.value:
    print(value)

Selected subset:
All


### Get total number of strains

In [19]:
query = """
MATCH (s:Strain)
WHERE s.taxonomyId = 'taxonomy:2697049' AND s.hostTaxonomyId = 'taxonomy:9606'
RETURN count(s)
"""

In [20]:
strains = graph.evaluate(query)

In [21]:
print('Total number of human strains:', strains)

Total number of human strains: 73188


### Get variants for selected protein

In [22]:
query = """
MATCH (g:Gene)-[:ENCODES]-(p:Protein{name: $protein_name})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})<-[:HAS_VARIANT]-(s:Strain)
WHERE s.hostTaxonomyId = 'taxonomy:9606'
WITH v.proteinPosition AS residue, count(v.proteinVariant) AS count, 
     v.proteinVariant + '(' + count(v.proteinVariant) + ')' AS variationId ,
     split(v.proteinVariant, ':')[1] + '(' + count(v.proteinVariant) + ')' AS annotation
     ORDER by count DESC
WITH residue, count, variationId, annotation
RETURN residue, collect(variationId) AS variationId, collect(annotation) AS annotation, sum(count) AS count ORDER BY residue
"""

### Add mutation annotation to each residue

In [23]:
variants = graph.run(query, protein_name=protein_name).to_data_frame()

In [24]:
variants['variationId'] = variants['variationId'].apply(lambda x: ', '.join(x))
variants['annotation'] = variants['annotation'].apply(lambda x: ', '.join(x))

In [25]:
variants['annotation'] = variants['annotation'].str.replace('p.', '')

Create a color scale based on the log mutation frequency

In [26]:
variants['scale'] = variants['count'].apply(np.log) / math.log(strains)

In [27]:
n_colors = 100
colors = cm.Reds(np.linspace(0.0, 1.0, n_colors))
col = np.empty(n_colors, dtype=object)

for i, color in enumerate(colors):
    col[i] = matplotlib.colors.rgb2hex(color)

In [28]:
variants['color'] = variants['scale'].apply(lambda x: col[round(x*n_colors)])

In [29]:
variants.head()

,annotation,count,residue,variationId,scale,color
0,2F>L(2),2,2,QHD43416.1:p.2F>L(2),0.061884,#ffebe2
1,"3V>G(18), 3V>F(2)",20,3,"QHD43416.1:p.3V>G(18), QHD43416.1:p.3V>F(2)",0.267457,#fcb499
2,4F>S(1),1,4,QHD43416.1:p.4F>S(1),0.000000,#fff5f0
3,"5L>F(673), 5L>I(1)",674,5,"QHD43416.1:p.5L>F(673), QHD43416.1:p.5L>I(1)",0.581498,#f34a36
4,"6V>F(55), 6V>I(1)",56,6,"QHD43416.1:p.6V>F(55), QHD43416.1:p.6V>I(1)",0.359381,#fc9576


### Get PDB structures for selected protein

In [30]:
query = """
MATCH (g:Gene)-[:ENCODES]->(p:Protein{name: $protein_name})-[h:HAS_TERTIARY_STRUCTURE]->(c:Chain)-[:IS_PART_OF_STRUCTURE]->(s:Structure)
RETURN p.name AS name, p.start, p.end, c.name AS structureChainId, c.uniprotStart, c.uniprotEnd, c.pdbStart, c.pdbEnd, s.resolution AS resolution, s.description AS description, h.coverage AS coverage
ORDER BY resolution, coverage DESC
"""

In [31]:
chains = graph.run(query, protein_name=protein_name).to_data_frame()

In [32]:
if description_widget.value[0] != 'All':
    print("Selecting subset:", description_widget.value)
    chains = chains.query(f'description in {description_widget.value}')
else:
    print('All')

All


In [33]:
chains['structureChainId'] = chains['structureChainId'].str[4:]

In [34]:
chains['structureId'] = chains['structureChainId'].str[:4]

In [35]:
chains.head()

,c.pdbEnd,c.pdbStart,c.uniprotEnd,c.uniprotStart,coverage,description,name,p.end,p.start,resolution,structureChainId,structureId
0,[966],[917],[966],[917],0.039277,spike protein,Spike glycoprotein,1273,1,1.50,6M1V.A,6M1V
1,"[359, 369, 384, 516]","[338, 366, 371, 392]","[359, 369, 384, 516]","[338, 366, 371, 392]",0.129615,"Spike protein S1, COVA2-39 heavy chain, COVA2-...",Spike glycoprotein,1273,1,1.71,7JMP.A,7JMP
2,[528],[334],[528],[334],0.153181,"Spike glycoprotein, Nanobody H11-D4",Spike glycoprotein,1273,1,1.80,6YZ5.E,6YZ5
3,"[518, 528]","[334, 520]","[518, 528]","[334, 520]",0.152396,"Spike protein S1, Heavy chain of B38, Light ch...",Spike glycoprotein,1273,1,1.84,7BZ5.A,7BZ5
4,[528],[334],[528],[334],0.153181,"Spike glycoprotein, H11-H4",Spike glycoprotein,1273,1,1.85,6ZBP.EEE,6ZBP


In [36]:
chains.drop_duplicates(subset=['structureChainId'], inplace=True)
chains.sort_values(by=['structureChainId'], inplace=True)

#### Map uniprot residue numbers to PDB residue numbers

In [37]:
def uniprot_to_pdb_mapping(row):
    mapping = dict()
    for (us,ue, ps, pe) in zip(row['c.uniprotStart'], row['c.uniprotEnd'], row['c.pdbStart'], row['c.pdbEnd']):
        ps = int(ps)
        pe = int(pe)
        if (ue-us != pe-ps):
            print('length mismatch:', row['structureChainId'], ue-us, pe-ps)
        else:
            offset = ps - us
            for v in range(us, ue+1):
                mapping[v] = offset + v
                
    #print(mapping)
    return mapping

In [38]:
chains['mapping'] = chains.apply(lambda row: uniprot_to_pdb_mapping(row), axis=1)

### Visualize mutation sites

Mutations are mapped onto protein chains for available 3D protein structures.

Display options:

|||
|:-|:-|
| *show_bio_assembly* | Toggle display of the biologically relevant quaternary structure |
| *show_surface* | Toggle surface for protein chain |
| *show_annotations* | Toggle display of mutation information<br>{PDBId}.{chainId}.{PDBResidue}: {UniProtResidue}{aminoAcid1}>{aminoAcid2}(# observations)<br>Example: 6Z43.A.614: 614D>G(58984), 614D>N(6) |
| *size* | Change size of visualization |
| *font* | Change font size of annotations |
| *logFreq* | Change minimum threshold to display mutations based on normalized log of mutation frequency [0.0 - 1.0]|
| *structure* | Move slider to browse through available structures |

In [39]:
# Setup viewer
def view_mutations(df, variants, *args):
    chainIds = list(df['structureChainId'])

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i): 
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer1
        viewer1 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer1.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.8}})

        # highlight chain of interest in blue
        viewer1.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        # non-polymer style
        viewer1.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False, 'colorscheme': 'greenCarbon'}})
        
        mapping = df['mapping'].iloc[i]

        for row in variants.itertuples():
            # get PDB residue mapping from a UniProt residue number
            res_num = mapping.get(row.residue, 0)
            col = row.color
            if res_num > 0 and row.scale > logFreq:
                mut_res = {'resi': res_num, 'chain': chain_id}
                viewer1.addStyle(mut_res, {'sphere':{'color':col, 'opacity': 1.0}}) 

                if show_short_label:
                    label = row.annotation
                if show_long_label:
                    label = chainIds[i] + "." + str(res_num) + ": " + row.variationId
                if show_short_label or show_long_label:
                    viewer1.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory', 'opacity': 1.0}, {'resi': res_num, 'chain': chain_id})

        description = df['description'].iloc[i]
        resolution = df['resolution'].iloc[i]
        coverage = df['coverage'].iloc[i]
        name = df['name'].iloc[i]
        
        print(name)
        print()
        print(f'PDB Id: {pdb_id}, chain Id: {chain_id}, resolution: {resolution}, sequence coverage: {coverage:.2f}')
        print(f'description: {description}')
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])

        viewer1.zoomTo({'chain': chain_id})
        viewer1.center({'chain': chain_id})
        
        if show_surface:
             viewer1.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})

        return viewer1.show()
       
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.5, min=0, max=1, step=0.05, description='logFreq:', 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, size=z_widget, font=f_widget, logFreq=l_widget, i=s_widget)

def view_image1():
    return viewer1.png()

In [40]:
view_mutations(chains, variants);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [41]:
# https://stackoverflow.com/questions/32468402/how-to-explode-a-list-inside-a-dataframe-cell-into-separate-rows
import copy

def pandas_explode(df, column_to_explode):
    """
    Similar to Hive's EXPLODE function, take a column with iterable elements, and flatten the iterable to one element 
    per observation in the output table

    :param df: A dataframe to explod
    :type df: pandas.DataFrame
    :param column_to_explode: 
    :type column_to_explode: str
    :return: An exploded data frame
    :rtype: pandas.DataFrame
    """

    # Create a list of new observations
    new_observations = list()

    # Iterate through existing observations
    for row in df.to_dict(orient='records'):

        # Take out the exploding iterable
        explode_values = row[column_to_explode]
        del row[column_to_explode]

        # Create a new observation for every entry in the exploding iterable & add all of the other columns
        for explode_value in explode_values.items():

            # Deep copy existing observation
            new_observation = copy.deepcopy(row)

            # Add one (newly flattened) value from exploding iterable
            new_observation[column_to_explode] = explode_value

            # Add to the list of new observations
            new_observations.append(new_observation)

    # Create a DataFrame
    return_df = pd.DataFrame(new_observations)

    # Return
    return return_df

### Expand chains into residues

In [42]:
residues = pandas_explode(chains, 'mapping')
residues['uniprotPosition'] = residues['mapping'].apply(lambda x: x[0])
residues['pdbPosition'] = residues['mapping'].apply(lambda x: x[1])
residues.drop(columns='mapping', inplace=True)

In [43]:
residues = residues.drop(columns=['c.pdbEnd', 'c.pdbStart', 'c.uniprotEnd', 'c.uniprotStart', 'p.end', 'p.start'])

In [44]:
residues.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition
0,0.026709,Spike protein S2,Spike glycoprotein,2.47,6LVN.A,6LVN,1169,2
1,0.026709,Spike protein S2,Spike glycoprotein,2.47,6LVN.A,6LVN,1170,3
2,0.026709,Spike protein S2,Spike glycoprotein,2.47,6LVN.A,6LVN,1171,4
3,0.026709,Spike protein S2,Spike glycoprotein,2.47,6LVN.A,6LVN,1172,5
4,0.026709,Spike protein S2,Spike glycoprotein,2.47,6LVN.A,6LVN,1173,6


In [45]:
variants = variants[['residue', 'variationId', 'annotation', 'scale', 'color']]

In [46]:
residues_variants = residues.merge(variants, left_on='uniprotPosition', right_on='residue')

In [47]:
residues_variants.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color
0,0.026709,Spike protein S2,Spike glycoprotein,2.47,6LVN.A,6LVN,1169,2,1169,"QHD43416.1:p.1169I>L(1), QHD43416.1:p.1169I>T(...","1169I>L(1), 1169I>T(1), 1169I>F(1)",0.098083,#fee5d8
1,0.027494,Spike protein S2,Spike glycoprotein,2.47,6LVN.B,6LVN,1169,2,1169,"QHD43416.1:p.1169I>L(1), QHD43416.1:p.1169I>T(...","1169I>L(1), 1169I>T(1), 1169I>F(1)",0.098083,#fee5d8
2,0.026709,Spike protein S2,Spike glycoprotein,2.47,6LVN.D,6LVN,1169,2,1169,"QHD43416.1:p.1169I>L(1), QHD43416.1:p.1169I>T(...","1169I>L(1), 1169I>T(1), 1169I>F(1)",0.098083,#fee5d8
3,0.273370,Spike glycoprotein,Spike glycoprotein,3.00,6XRA.A,6XRA,1169,1169,1169,"QHD43416.1:p.1169I>L(1), QHD43416.1:p.1169I>T(...","1169I>L(1), 1169I>T(1), 1169I>F(1)",0.098083,#fee5d8
4,0.273370,Spike glycoprotein,Spike glycoprotein,3.00,6XRA.B,6XRA,1169,1169,1169,"QHD43416.1:p.1169I>L(1), QHD43416.1:p.1169I>T(...","1169I>L(1), 1169I>T(1), 1169I>F(1)",0.098083,#fee5d8


In [48]:
residues_variants.to_csv(output_file_name, index=False)

## Now run the next step
Map mutations occuring at protein-protein interaction sites: [2-MapToPolymerInteractions.ipynb](2-MapToPolymerInteractions.ipynb)